Extracting User's Google Calendar

In [153]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
import datetime


In [154]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [155]:
import methods #file containing functions to interact with Google calendar like add event, create event, etc
import json
import re

google_calendar = methods.get_google_calendar_service() #get the google calendar service
tasks = methods.get_task_service() #get the task service
tasks_list = methods.extractTasks(tasks)
calendar = methods.extractCalendar(google_calendar) #extract the calendar from the service
client = methods.set_up_ChatGPT(calendar, tasks_list) #set up the ChatGPT model
now = methods.current_time()

To-Do list for the week:
1. GPT prompt engineering - make GPT refine prompt
2. Ask COE to mail degree certificate to Umich
3. Jacket and bag zip repair
4. Send emails to professors for research in summer

Events for today:
- SI 649 - DIS from 4:00 PM to 5:30 PM

Logging Previous Conversation

In [156]:
def log_activity(role, message):
    filename = f"activity_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [157]:
def log_message(role, message):
    filename = f"conversation_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [158]:
def get_past_conversation():
    filename = f"conversation_log.txt"
    try:
        with open(filename, "r") as file:
            return file.read()
    except FileNotFoundError:
        return ""

Get User's Query

In [159]:
past_conversation = get_past_conversation()
# schedule tasks or calendar events

context = f"Here's my calendar and To-Do list for the week, store it for your reference. Calendar: {calendar}\n To-Do Tasks: {tasks_list} Today's date and time is: {now}\n"
instruction = """\nAs my virtual assistant, you are an expert with managing the calendar and scheduling tasks. 
First, you can create, move, check, list, update, delete, and handle recurring events based on the user's requests.
When the user doesn't otherwise provide enough information, you can ask for more details, upto 4 times. Otherwise use your best judgement.
When the user wants to create an event or move an event and if the user hasn't specified when they want it scheduled, then refer to the calendar and use your best judgement to schedule the event. Keep in mind when the user would normally eat or sleep. Assume default duration for any event as 1 hour.
Check for any conflicts before scheduling an event.
Always ask for confirmation before making any changes to the calendar, if the user doesn't agree, then generate another suitable time slot.
When you are done, please let the user know that the task has been completed, and don't ask any more questions.
Second, you can help me manage my to-do list by using your own judgement and scheduling the things on it onto my calendar.
When the user wants to schedule their to-do tasks, reorder the tasks by priority using your judgement and output all the tasks with proposed timeslots for each task. Create a balanced schedule.
Check if a task is already in the calendar. 
Take your time to think and plan the schedule before presenting it to the user.
Also use your best guess to determine the length of the tasks. Assume that all the tasks have to be scheduled.
If the user agrees, then add the events to the calendar and let the user know that the tasks have been scheduled.
If the user doesn't agree, then regenerate the timeslots for the tasks and ask for confirmation again. 
When you are done, let the user know that you have completed your task. Unless you are done with the task, always end with a question mark.
Please perform the necessary actions using the Google Calendar API.\n
Example input: 'schedule all my tasks'
Example output: 'Sure! Here's a proposed schedule for all your tasks. Is there anything you'd like to change?
- Task 1: Apr 2nd, 10:00 AM - 11:00 AM
- Task 2: Apr 2nd, 11:00 AM - 12:00 PM
- Task 3: Apr 2nd, 12:00 PM - 1:00 PM\n
Here's the previous conversation with the user, read it and understand user preferences if any, use it to make your best judgement on scheduling.\n"""

# initial setup

initial_prompt = context + instruction + past_conversation

2024-04-17T17:48:28.509201 | user: schedule me a jog for when I'm free
2024-04-17T17:48:32.196634 | assistant: I see that you have some free time today after your SI 649 - DIS class at 5:30 PM. Would you like to schedule a jog for after that, around 6:00 PM?
2024-04-17T17:48:38.217797 | user: yes
2024-04-17T17:48:39.141651 | assistant: Great! I have scheduled a jog for you today at 6:00 PM. Your calendar is now updated. Is there anything else you'd like to schedule or modify?
2024-04-17T17:48:42.896164 | user: no
2024-04-17T17:48:43.450716 | assistant: Task completed. Let me know if you need any further assistance.
2024-04-17T17:50:33.775964 | user: reschedule my jog to tomorrow 
2024-04-17T17:50:35.316351 | assistant: I see that you have some free time tomorrow after your EECS 545 project meeting at 2:30 PM. Would you like to schedule the jog for after that, around 3:30 PM?
2024-04-17T17:50:47.979421 | user: yes
2024-04-17T17:50:49.058129 | assistant: Great! I have rescheduled your jo

In [160]:
user_request = input()
combined_prompt = initial_prompt + "\nUser request: " + user_request


In [161]:
def chat_gpt(client, messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=500
    )
    return response.choices[0].message.content

messages = [
    {
        "role": "system",
        "content": initial_prompt
    },
    {
        "role": "user",
        "content": user_request
    }
]
log_activity("system", context+instruction)
log_activity("user", user_request)

log_message("user", user_request)

In [162]:
finalized_response = False

#establishing feedback loop to chat with user
while not finalized_response:
    response = chat_gpt(client, messages)
    log_activity("assistant", response)
    print(response)
    log_message("assistant", response)
    messages.append({
        "role": "assistant",
        "content": response
    })
    if re.compile(r'\?').search(response):
        finalized_response = False
        user_feedback = input()
        messages.append({
            "role": "user",
            "content": user_feedback
        })
        log_message("user", user_feedback)
        log_activity("user", user_feedback)
    else:
        finalized_response = True


Sure! Let's organize your to-do list based on priority and schedule them onto your calendar for this week. Here's a proposed schedule for your tasks:

1. **Jacket and bag zip repair**:
   - Proposed Time: April 22nd, 11:30 AM - 12:30 PM

2. **Send emails to professors for research in summer**:
   - Proposed Time: April 22nd, 1:00 PM - 2:00 PM

3. **Ask COE to mail degree certificate to Umich**:
   - Proposed Time: April 22nd, 9:00 AM - 10:00 AM

4. **GPT prompt engineering - make GPT refine prompt**:
   - Proposed Time: April 22nd, 10:30 AM - 11:30 AM

Would you like me to add these tasks to your calendar as proposed?
Sure! Let's organize your to-do list based on priority and schedule them onto your calendar for this week. Here's a proposed schedule for your tasks:

1. **Jacket and bag zip repair**:
   - Proposed Time: April 22nd, 11:30 AM - 12:30 PM

2. **Send emails to professors for research in summer**:
   - Proposed Time: April 22nd, 1:00 PM - 2:00 PM

3. **Ask COE to mail degree 

In [163]:
#get GPT to output the function calls I need to make to the calendar API
instruction2 = """\nBased on the user's request you've just processed, please output the appropriate function call(s) to interact with the Google Calendar API. Format your response as a single string containing one or more function calls separated by semicolons. Do not include explanations or additional text.
All dates should be in 'YYYY-MM-DD' format, and times should be in 24-hour format 'HH:MM'. Additionally, assume default duration as 1 hour for any event. Use createEvent for scheduling tasks on the to-do list.
Available function templates to include in the response string:
- createEvent(description, date, time, duration)
- moveEvent(event_id, new_date, new_time)
- updateEvent(event_id, new_color, new_duration, new_location)
- deleteEvent(event_id)
- createRecurringEvent(description, start_date, time, duration, frequency, until_date). Frequency can have the value of DAILY, WEEKLY, MONTHLY, YEARLY.
- deleteRecurringEvent(recurring_event_id) 
- updateRecurringEvent(recurring_event_id, new_color, new_duration, new_location, new_frequency, new_until)
For all functions other than createEvent and createRecurringEvent, make sure to fill in the id of the event you are referring to from the calendar provided.
Make sure to fill in the required information, such as event names, dates, times, etc., based on the user's request context. Replace placeholders like 'today' or 'tomorrow' with specific dates in the format YYYY-MM-DD. If more details are needed from the user for a function call, use your best judgment to complete the function with the information available.
Example input1: 'Schedule a team meeting for Friday afternoon at 3pm'
Example output1: 'createEvent("Team Meeting", "YYYY-MM-DD", "15:00", 1:00)'
Example input2: 'Update my team meeting to be two hours'
Example output2: 'updateEvent("event_id", '', 2:00, '')
Example input3: 'Create a recurring meeting every Wednesday at 2pm until the end of June'
Example output3: 'createRecurringEvent("Meeting", "YYYY-MM-DD", "02:00", "1:00", "WEEKLY", "YYYY-MM-DD")
Example input4: 'Update my recurring meeting every Wednesday at 2pm to be every month until the end of August'
Example output4: 'updateRecurringEvent("RecurringEventId", "", "02:00", "1:00", "", "MONTHLY", "YYYY-MM-DD")
Example input5: 'Move my team meeting to tomorrow at 12pm'
Example output5: 'moveEvent("event_id", 'YYYY-MM-DD', '12:00')'
Example input6: 'Clear my afternoon'
Example output7: 'deleteEvent("event_id");deleteEvent("event_id")'
Example input8: 'Schedule all my tasks'
Example output8: 'createEvent("Task 1", "YYYY-MM-DD", "10:00", 1:00);createEvent("Task 2", "YYYY-MM-DD", "11:00", 1:00);createEvent("Task 3", "YYYY-MM-DD", "12:00", 1:00)\n

Now, please generate the necessary function call(s) based on the user's request."""

In [164]:
messages.append({
    "role": "system",
    "content": instruction2
})
log_activity("system", instruction2)
function_call = chat_gpt(client, messages)
log_activity("assistant", function_call)
print(function_call)
#Example of function call: createEvent("Meeting with John", "2022-12-31", "10:00 AM", 2)

createEvent("Jacket and bag zip repair", "2024-04-22", "11:30", 1:00);createEvent("Send emails to professors for research in summer", "2024-04-22", "13:00", 1:00);createEvent("Ask COE to mail degree certificate to Umich", "2024-04-22", "09:00", 1:00);createEvent("GPT prompt engineering - make GPT refine prompt", "2024-04-22", "10:30", 1:00)


In [165]:
if '(' in function_call:
    function_calls = [line.strip() for line in function_call.split(';')]
    print(function_calls)

    # Define a dictionary mapping function names to their corresponding Python functions
    functions = {
        "createEvent": methods.createEvent,
        "moveEvent": methods.moveEvent,
        "updateEvent": methods.updateEvent,
        "deleteEvent": methods.deleteEvent,
        "createRecurringEvent": methods.createRecurringEvent,
        "updateRecurringEvent": methods.updateRecurringEvent,
        "deleteRecurringEvent": methods.deleteRecurringEvent,
    }

    # Iterate through the function calls
    for call in function_calls:
        # Split the string to extract function name and arguments
        if call != '':
            parts = call.split('(')
            function_name = parts[0]
            arguments = parts[1].rstrip(')').split(', ')

        # Call the corresponding function with the provided arguments
        if function_name in functions:
            functions[function_name](*arguments)
        else:
            print(f"Function '{function_name}' not found.")

['createEvent("Jacket and bag zip repair", "2024-04-22", "11:30", 1:00)', 'createEvent("Send emails to professors for research in summer", "2024-04-22", "13:00", 1:00)', 'createEvent("Ask COE to mail degree certificate to Umich", "2024-04-22", "09:00", 1:00)', 'createEvent("GPT prompt engineering - make GPT refine prompt", "2024-04-22", "10:30", 1:00)']
Create event called
The data and time is  2024-04-22 / 11:30
UTC start time:  2024-04-22 15:30:00+00:00 UTC end time:  2024-04-22 16:30:00+00:00
Event created: Jacket and bag zip repair on 2024-04-22 at 11:30 for duration of 1:00 hours.
Create event called
The data and time is  2024-04-22 / 13:00
UTC start time:  2024-04-22 17:00:00+00:00 UTC end time:  2024-04-22 18:00:00+00:00
Event created: Send emails to professors for research in summer on 2024-04-22 at 13:00 for duration of 1:00 hours.
Create event called
The data and time is  2024-04-22 / 09:00
UTC start time:  2024-04-22 13:00:00+00:00 UTC end time:  2024-04-22 14:00:00+00:00
Ev